In [94]:
import pandas as pd
import numpy as np

In [95]:
tweets_data = pd.read_csv('tweets.csv')

In [96]:
tweets_data.shape

(7920, 3)

In [97]:
# from google.colab import drive
# drive.mount('/content/drive')

In [98]:
tweets_data.columns

Index(['id', 'label', 'tweet'], dtype='object')

In [99]:
tweets_data.head(15)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
5,6,1,iPhone software update fucked up my phone big ...
6,7,0,Happy for us .. #instapic #instadaily #us #son...
7,8,0,New Type C charger cable #UK http://www.ebay.c...
8,9,0,Bout to go shopping again listening to music #...
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...


In [100]:
tweets_data['tweet'][4]

"What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!"

In [101]:
# tweets_data['label'][1]

In [102]:
tweets_data['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [103]:
tweets_data=tweets_data.drop('id',axis=1)
X=tweets_data.drop('label',axis=1)
y=tweets_data['label']

In [104]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [105]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [106]:
from bs4 import BeautifulSoup
import re
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text


#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\^^", "", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

def lemmatize_text(text):
    lem=nltk.stem.wordnet.WordNetLemmatizer()
    text= ' '.join([lem.lemmatize(word) for word in text.split()])
    return text
    

In [171]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

class DummyTransformer(BaseEstimator,TransformerMixin):
    
    '''
    dummy class to inherit from to avoid typing the fit method for everything
    '''
    def __init__(self):
        return None
    def fit(self,X=None,y=None):
        return self
    def transform(self,X=None):
        return X
    
def do_basic_text_preprocessing(text:str):
    preprocessed_text=denoise_text(text)
    preprocessed_text=remove_special_characters(preprocessed_text)
    # preprocessed_text=lemmatize_text(preprocessed_text)

    return preprocessed_text

class TextPreprocessor(DummyTransformer):
    def transform(self,X:pd.DataFrame):
#         preprocessed_X_np_array=X.apply(do_basic_text_preprocessing).values
        # preprocessed_X_nparr=X['tweet'].apply(lambda x:do_basic_text_preprocessing(x)).values
        preprocessed_X_df=X['tweet'].apply(lambda x:do_basic_text_preprocessing(x))
        return preprocessed_X_df

class Tokenizer(DummyTransformer):
    def transform(self, X):
        X=pd.DataFrame(X)
        # print(X.info())
        # print(X)
        X_tokenized=X['tweet'].apply(lambda x:simple_preprocess(x))
        # X_tokenized=X.apply(word_tokenize,axis=0) ## requires downloading punkt
        # print(X_tokenized)
        return X_tokenized

class SparseToDenseArr(DummyTransformer):
    def transform(self,X=None):
#         preprocessed_X_np_array=X.apply(do_basic_text_preprocessing).values
        dense_arr=X.toarray()

        return dense_arr    

class MetaFeatureEngineer(DummyTransformer):
    def transform(self, X=None,y=None):
        return None

tfidf_meta_union_inst=FeatureUnion([('tfidf',TfidfVectorizer()),('metafeature',MetaFeatureEngineer())])
    
tfidf_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tfidf_meta_union_inst',tfidf_meta_union_inst),('sparsetodense',SparseToDenseArr())])

# tfidf_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tfidf',TfidfVectorizer()),('sparsetodense',SparseToDenseArr())])


bow_meta_union_inst=FeatureUnion([('bow',CountVectorizer()),('metafeature',MetaFeatureEngineer())])
    
bow_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('bow_meta_union_inst',bow_meta_union_inst)])




In [172]:
### word2vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

class CustomWord2VecTransformer(DummyTransformer):
    # def __init__(self,vector_size = 300, window = 6 , min_count = 3, sg = 0):
    def __init__(self,**kwargs):
        self.input_args=kwargs
        # self.w2v_model=None

        # allowed_keys=['vector_size','window','min_count','sg']
        # self.__dict__.update((k, v) for k, v in kwargs.items() if k in allowed_keys)



    def fit(self,X,y=None):
        self.w2v_model=Word2Vec(X,**self.input_args)

        return self

    def get_mean_embedding_for_doc(self,doc_tokens):
        model=self.w2v_model   
        embeddings=[]
        for tok in doc_tokens:
            if tok in model.wv.index_to_key:
                embeddings.append(model.wv.get_vector(tok))
        return np.mean(embeddings,axis=0)

    
    def transform(self, X):

        X_transformed=X.apply(lambda x:self.get_mean_embedding_for_doc(x))
        X_transformed=pd.DataFrame(X_transformed.tolist())
        # print(X_transformed)
        return X_transformed


class GoogleWord2VecTransformer(DummyTransformer):
    # def __init__(self,vector_size = 300, window = 6 , min_count = 3, sg = 0):
    def __init__(self,*args,**kwargs):
        self.w2vmodel=KeyedVectors.load_word2vec_format(*args,**kwargs)

        self.input_args=kwargs
        # allowed_keys=['vector_size','window','min_count','sg']
        # self.__dict__.update((k, v) for k, v in kwargs.items() if k in allowed_keys)


    def fit(self,X,y=None):
        
        return self

    def get_mean_embedding_for_doc(self,doc_tokens):
        model=self.w2vmodel
        embeddings=[]
        for tok in doc_tokens:
            if tok in model: ## note the synatx to get vector etc. are different from that for gensim model
                embeddings.append(model[tok])
        return np.mean(embeddings,axis=0)


    def transform(self, X):
        X_transformed=X.apply(lambda x:self.get_mean_embedding_for_doc(x))
        X_transformed=pd.DataFrame(X_transformed.tolist())
        return X_transformed




In [173]:

# google_meta_union_inst=FeatureUnion([
#     ('google',GoogleWord2VecTransformer("/content/drive/MyDrive/ml-ai files arjun/GoogleNews-vectors-negative300.bin",binary=True)),
#     ('metafeature',MetaFeatureEngineer())])

# google_trans_inst=GoogleWord2VecTransformer("/content/drive/MyDrive/ml-ai files arjun/GoogleNews-vectors-negative300.bin",binary=True)


In [174]:
# google_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tokenizer',Tokenizer()),('google',google_trans_inst())])
custom_w2v_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tokenizer',Tokenizer()),('customw2v',CustomWord2VecTransformer())])


In [175]:

X_train_tfidf=custom_w2v_pipeline.fit_transform(X_train)
X_test_tfidf=custom_w2v_pipeline.transform(X_test)

# X_train=TextPreprocessor().transform(X_train)

<ipython-input-106-528d8e3112de>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


            0         1         2         3         4         5         6   \
0    -0.101155  0.293836 -0.009673  0.109925  0.202355 -0.369149  0.028533   
1     0.222628  0.349493 -0.427015  0.484218  0.215723 -1.733066  0.001228   
2    -0.166048  0.360263  0.006424  0.135612  0.281680 -0.422261  0.014386   
3    -0.046643  0.502848  0.013319  0.180812  0.241929 -0.612826  0.155780   
4    -0.164313  0.410249  0.013402  0.153985  0.302769 -0.481192  0.040237   
...        ...       ...       ...       ...       ...       ...       ...   
6331 -0.164654  0.443424  0.138773  0.150636  0.239597 -0.423581  0.082000   
6332 -0.067155  0.385732  0.005339  0.144759  0.180914 -0.444660  0.114349   
6333 -0.199630  0.425910  0.024032  0.157504  0.334407 -0.487582  0.011459   
6334  0.027704  0.271478 -0.041143  0.140804  0.093746 -0.484550  0.092072   
6335  0.083316  0.336840 -0.025756  0.145630  0.040891 -0.521666  0.146156   

            7         8         9   ...        90        91    

<ipython-input-106-528d8e3112de>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


            0         1         2         3         4         5         6   \
0     0.069610  0.483735 -0.056395  0.229231  0.116981 -0.791935  0.176551   
1    -0.083927  0.322315  0.031219  0.131417  0.173827 -0.395406  0.067908   
2    -0.052664  0.349686  0.027953  0.133387  0.169093 -0.412658  0.097112   
3    -0.126973  0.436715  0.001226  0.161286  0.250006 -0.504830  0.076913   
4    -0.107375  0.320135  0.008699  0.138838  0.230793 -0.439413  0.026094   
...        ...       ...       ...       ...       ...       ...       ...   
1579 -0.066843  0.441214  0.069046  0.172105  0.141570 -0.451384  0.184120   
1580 -0.121269  0.364146  0.065761  0.141776  0.227400 -0.392874  0.060282   
1581 -0.076825  0.373910  0.072218  0.132286  0.175032 -0.375872  0.108198   
1582  0.015625  0.519374 -0.018821  0.168604  0.132834 -0.598176  0.225431   
1583 -0.060434  0.438656  0.051149  0.180651  0.166148 -0.483273  0.164463   

            7         8         9   ...        90        91    

In [176]:
print(np.sum(X_train_tfidf >0))

0     1130
1     6336
2     4320
3     6336
4     6330
      ... 
95    6258
96    5701
97       0
98    2335
99     442
Length: 100, dtype: int64


In [177]:
### pass parameters to Pipeline steps

In [178]:
from importlib import import_module
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 



# algos=[('naive_bayes','GaussianNB'),('tree','DecisionTreeClassifier'),('linear_model','LogisticRegression')]
# algos=[('linear_model','LogisticRegression')] ### why did i even do this?

result_dict={}
X_train_to_use=X_train_tfidf
X_test_to_use=X_test_tfidf
y_train_to_use=y_train
y_test_to_use=y_test

algo_insts=[GaussianNB(),DecisionTreeClassifier(),LogisticRegression()]

for algo_inst in algo_insts:
    algo_inst.fit(X_train_to_use, y_train_to_use)
    y_pred=algo_inst.predict(X_test_to_use)
    result_dict.update({f'{type(algo_inst).__name__}':{'accuracy':accuracy_score(y_pred,y_test_to_use)}})
pd.DataFrame(result_dict)


# for algo in algos:
#     ## apparently the methods of the module are attributes of the obj outputted by import_module
#     class_var=getattr(import_module(f'sklearn.{algo[0]}'),algo[1]) 
#     clf=class_var()
#     # clf=module.
#     clf.fit(X_train_to_use, y_train_to_use)
#     y_pred=clf.predict(X_test_to_use)
#     result_dict.update({f'{type(clf).__name__}':{'accuracy':accuracy_score(y_pred,y_test_to_use)}})
# pd.DataFrame(result_dict)

,GaussianNB,DecisionTreeClassifier,LogisticRegression
accuracy,0.677399,0.613636,0.744949


In [179]:
#### preprocess the text to that is in the required format for word2vec



In [180]:
# print(simple_preprocess(tweets_data['tweet'][1])[:30])

In [ ]:
### google accuracy 0.8118686868686869
### cbow accuracy 0.8377525252525253
